In [8]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

1.11.0
0.15.2a0


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import tensorflow as tf
import os
gpus = tf.config.experimental.list_physical_devices('GPU')
# print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    print(gpu)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [3]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# root_input_dir = '/kaggle/input'
# train_data_dir = validation_data_dir = test_data_dir = None

# train_data_dir = f'{root_input_dir}/sohas-wepon-classification/Sohas_weapon-Classification/'
# # validation_data_dir = f'{root_dir}/intel-image/val'
# # test_data_dir = f'{root_dir}/intel-image/val'

# def get_data(ims=150, batch_size=32):
    
#     augdict = {
#         'rotation_range':20,
#         'width_shift_range':0.2,
#         'height_shift_range':0.2,
#         'shear_range':0.2,
#         'zoom_range':0.2,
#         'horizontal_flip':True,
#         'fill_mode':'nearest',
#     }

#     split = 0.2

#     datagen = ImageDataGenerator(
#     #     rescale=1.0 / 255,
#         # **augdict,
#         validation_split=split
#     )

#     auggen = ImageDataGenerator(
#     #     rescale=1.0 / 255,
#         **augdict,
#         validation_split=split
#     )

#     valgen = ImageDataGenerator(
#     #     rescale=1.0 / 255,
#         # **augdict,
#         validation_split=split
#     )
#     testgen = ImageDataGenerator(
#     #     rescale=1.0 / 255,
#         # **augdict,
#     )


#     print(end='data_generator:\t')
#     data_generator = None
#     data_generator = datagen.flow_from_directory(
#         train_data_dir,
#         target_size=(ims, ims),
#         batch_size=batch_size,
#         class_mode='categorical',
#         subset='training',  # Specify 'training' or 'validation'
#         shuffle=True
#     )
    
#     print(end='aug_generator:\t')
#     aug_generator = None
#     aug_generator = auggen.flow_from_directory(
#         train_data_dir,
#         target_size=(ims, ims),
#         batch_size=batch_size,
#         class_mode='categorical',
#         subset='training',  # Specify 'training' or 'validation'
#         shuffle=True
#     )

#     print(end='val_generator:\t')
#     val_generator = None
#     val_generator = datagen.flow_from_directory(
#         train_data_dir,
#         target_size=(ims, ims),
#         batch_size=batch_size,
#         class_mode='categorical',
#         subset='validation',  # Specify 'training' or 'validation'
#         shuffle=False
#     )

#     test_generator = None
#     if test_data_dir:
#         print(end='test_generator:\t')
#         test_generator = testgen.flow_from_directory(
#             test_data_dir,
#             target_size=(ims, ims),
#             batch_size=batch_size,
#             class_mode='categorical',
#             shuffle=False
#         )

#     num_classes, nb_train_samples, nb_validation_samples = val_generator.num_classes, len(data_generator.classes), len(val_generator.classes)
#     return data_generator, aug_generator, val_generator, test_generator, ims, batch_size, num_classes, nb_train_samples, nb_validation_samples


# data_generator, aug_generator, val_generator, test_generator, ims, batch_size, num_classes, nb_train_samples, nb_validation_samples = get_data(224, 32)
# # nb_train_samples, nb_validation_samples

# Dataset

In [10]:
# !pip install supervision
import supervision as sv

train_dir = 'obj_train_data/images/train/'
train_ann_dir = 'obj_train_data/labels/train/'
val_dir = 'obj_train_data/images/test/'
val_ann_dir = 'obj_train_data/labels/test/'


dataset = sv.DetectionDataset.from_yolo(
    images_directory_path=train_dir,
    annotations_directory_path=train_ann_dir,
    data_yaml_path='dataset.yaml')

len(dataset)

: 

: 

In [ ]:
import supervision as sv

SAMPLE_SIZE =16
SAMPLE_GRID_SIZE = (4,4)
SAMPLE_PLOT_SIZE = (16,16)

image_names = list(dataset.images.keys())[:SAMPLE_SIZE]

mask_annotator = sv.MaskAnnotator()
box_annotator = sv.BoxAnnotator()

images = []
for image_name in image_names:
#     image_name = image_name.split('/')[-1]
    image = dataset.images[image_name]
    annotations = dataset.annotations[image_name]
    labels = [
        dataset.classes[class_id]
        for class_id
        in annotations.class_id]
    annotates_image = mask_annotator.annotate(
        scene=image.copy(),
        detections=annotations)
    annotates_image = box_annotator.annotate(
        scene=annotates_image,
        detections=annotations,
        labels=labels)
    images.append(annotates_image)

sv.plot_images_grid(
    images=images,
    titles=image_names,
    grid_size=SAMPLE_GRID_SIZE,
    size=SAMPLE_PLOT_SIZE)

In [8]:
from torchvision.datasets import VisionDataset
from torchvision import transforms

class YOLOv5Dataset(VisionDataset):
    def __init__(self, train_dir, annotation_txt_dir, yaml_dir, transform=None):
        super(YOLOv5Dataset, self).__init__(train_dir, transform=transform)
        self.annotation_txt_dir = annotation_txt_dir

        # Load YAML file to get class names
        with open(os.path.join(yaml_dir, 'dataset.yaml'), 'r') as file:
            dataset_config = yaml.safe_load(file)
        self.class_names = dataset_config['names']

    def __getitem__(self, index):
        # Load image
        img_path = self.samples[index][0]
        img = Image.open(img_path).convert('RGB')

        # Load YOLOv5 format annotations
        annotation_path = os.path.join(self.annotation_txt_dir, os.path.basename(img_path).replace('.jpg', '.txt'))
        with open(annotation_path, 'r') as file:
            lines = file.readlines()

        # Process and parse annotations here to get bounding boxes and class labels
        # You will need to implement the parsing logic to match your annotation format

        # Apply transformations if needed
        if self.transform is not None:
            img = self.transform(img)

        # Return image and target (bounding boxes, labels)
        return img, target

    def __len__(self):
        return len(self.samples)


/Users/jijnasu/miniforge3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/jijnasu/miniforge3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: <E736BB25-B038-3AFE-BCF3-8026B5D2C7B6> /Users/jijnasu/miniforge3/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <7A536E8A-60F4-39F0-AE7A-79B04AC14B92> /Users/jijnasu/miniforge3/lib/python3.10/site-packages/torch/lib/libc10.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [9]:
import os
from torchvision import transforms
from torch.utils.data import DataLoader

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create an instance of your custom YOLOv5Dataset
train_dataset = YOLOv5Dataset(train_dir, annotation_txt_dir, yaml_dir, transform=transform)

# Create a DataLoader for the training dataset
batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)


NameError: name 'train_dir' is not defined

In [1]:
from ultralytics import YOLO

model = YOLO("yolov8m.pt")

In [2]:
results = model.predict('/Users/jijnasu/Workspace/OD-WeaponDetection/Weapons and similar handled objects/Sohas_weapon-Detection-YOLOv5/obj_train_data/images/train/ABbframe00166.jpg')


image 1/1 /Users/jijnasu/Workspace/OD-WeaponDetection/Weapons and similar handled objects/Sohas_weapon-Detection-YOLOv5/obj_train_data/images/train/ABbframe00166.jpg: 384x640 1 person, 1 couch, 1 potted plant, 1 cell phone, 323.1ms
Speed: 2.1ms preprocess, 323.1ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)


In [5]:
print(len(results))

1


In [6]:
result = results[0]

In [8]:
box = result.boxes[0]
print("Object type:", box.cls)
print("Coordinates:", box.xyxy)
print("Probability:", box.conf)

Object type: tensor([0.])
Coordinates: tensor([[ 771.6041,  191.9480, 1067.5435, 1075.1079]])
Probability: tensor([0.9408])


In [11]:
cords = box.xyxy[0].tolist()
class_id = box.cls[0].item()
conf = box.conf[0].item()
print("Object type:", class_id)
print("Coordinates:", cords)
print("Probability:", conf)

Object type: 0.0
Coordinates: [771.6040649414062, 191.947998046875, 1067.54345703125, 1075.10791015625]
Probability: 0.9407950043678284


In [12]:
cords = box.xyxy[0].tolist()
cords = [round(x) for x in cords]
class_id = result.names[box.cls[0].item()]
conf = round(box.conf[0].item(), 2)
print("Object type:", class_id)
print("Coordinates:", cords)
print("Probability:", conf)

Object type: person
Coordinates: [772, 192, 1068, 1075]
Probability: 0.94


In [13]:
for box in result.boxes:
  class_id = result.names[box.cls[0].item()]
  cords = box.xyxy[0].tolist()
  cords = [round(x) for x in cords]
  conf = round(box.conf[0].item(), 2)
  print("Object type:", class_id)
  print("Coordinates:", cords)
  print("Probability:", conf)
  print("---")

Object type: person
Coordinates: [772, 192, 1068, 1075]
Probability: 0.94
---
Object type: potted plant
Coordinates: [120, 33, 485, 791]
Probability: 0.66
---
Object type: couch
Coordinates: [1, 746, 419, 1074]
Probability: 0.6
---
Object type: cell phone
Coordinates: [734, 399, 806, 488]
Probability: 0.34
---


In [1]:
!cat dataset.yaml

# train: sohas-weapon-detection-yolov5/images/images/train/
# val: sohas-weapon-detection-yolov5/images/images/test/
train: train/images/
val: test/images/

# number of classes
nc: 6

# class names
names: ['pistol', 'smartphone', 'knife', 'monedero', 'billete', 'tarjeta']


In [1]:
from ultralytics import YOLO

model = YOLO("yolov8m.pt")

In [6]:
model.train(data="dataset.yaml", epochs=2, device='mps')

Ultralytics YOLOv8.0.180 🚀 Python-3.10.6 torch-2.0.1 MPS (Apple M1)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=dataset.yaml, epochs=2, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False